In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns

In [2]:
plt.rcParams["figure.figsize"]=[15,5]

In [21]:
raw_pp=pd.read_csv('out_pickandplace1.csv', usecols=['timestamp', 'data.ax', 'data.az'])
raw_pp.sort_values(by=['timestamp'], inplace=True)
raw_pp.reset_index(inplace=True)
raw_pp.dropna(axis=0, inplace=True)
raw_pp['timestamp'] =  pd.to_datetime(raw_pp['timestamp'])

In [22]:
raw_pp

,index,timestamp,data.ax,data.az
0,65437,2019-01-01 18:29:59.808471,0.018433,0.082947
1,65438,2019-01-01 18:29:59.809549,0.014771,0.075562
2,65439,2019-01-01 18:29:59.810583,0.013062,0.079468
3,65440,2019-01-01 18:29:59.811605,0.017639,0.082520
4,65441,2019-01-01 18:29:59.812619,0.014832,0.081970
5,65442,2019-01-01 18:29:59.813656,0.015808,0.083130
6,65443,2019-01-01 18:29:59.814662,0.017029,0.082031
7,65444,2019-01-01 18:29:59.815705,0.015198,0.081482
8,65445,2019-01-01 18:29:59.845527,0.016235,0.079468
9,65446,2019-01-01 18:29:59.851279,0.015259,0.083496


In [14]:
#Load LD data and plot
raw_pp["acc"] = (raw_pp["data.ax"]**2 + raw_pp["data.az"]**2) ** 0.5
acc=raw_pp[["timestamp", "acc"]]
test_pp=acc
# test_pp['acc_rolling_sum']=pd.Series.to_frame(test_pp.acc.rolling(100*10, center=True).sum())
test_pp['acc_rolling_std']=pd.Series.to_frame(test_pp.acc.rolling(100*2, center=True).std())


KeyError: 'data.ax'

In [5]:
%matplotlib qt

In [6]:
# plt.plot(test_pp['timestamp'], test_pp['acc'])
# plt.plot(test_pp['timestamp'], test_pp['acc_rolling_std'])

In [7]:
test_pp['detections']=pd.Series.to_frame(test_pp.acc_rolling_std.rolling(100, center=True).sum())

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
plt.plot(test_pp['timestamp'], test_pp['acc']*100)
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [9]:
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [10]:
test_pp['boards']=0
test_pp['boards'][test_pp['detections']>=2]=1

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
plt.plot(test_pp["timestamp"],test_pp["boards"])

In [12]:
working_times_raw=scipy.signal.find_peaks(test_pp.boards, distance=40*72, width=1)

In [13]:
working_times_raw_df=pd.DataFrame({"sample_number":working_times_raw[0], "working_time":working_times_raw[1]['widths']/72})

In [14]:
for x, row in working_times_raw_df.iterrows():
    working_times_raw_df.ix[x,'start_time']=test_pp.iloc[int(working_times_raw[1]['left_ips'][x])].timestamp
    working_times_raw_df.ix[x,'end_time']=test_pp.iloc[int(working_times_raw[1]['right_ips'][x])].timestamp
    working_times_raw_df.ix[x,'timestamp']=test_pp.iloc[int(working_times_raw[0][x])].timestamp

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [15]:
working_times_df=working_times_raw_df[working_times_raw_df.working_time>30]

In [16]:
working_times_df['event']=1

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
raw_pp_current=pd.read_csv('out_pickandplace1.csv', usecols=['timestamp', 'data.A1', 'data.A2', 'data.A3'])
raw_pp_current.dropna(axis=1, inplace=True)
raw_pp_current.sort_values(by=['timestamp'])
raw_pp_current.reset_index(inplace=True)
raw_pp_current['total_current']=raw_pp_current['data.A1']+raw_pp_current['data.A2']+raw_pp_current['data.A3']
raw_pp_current['timestamp']=pd.to_datetime(raw_pp_current['timestamp'])
raw_pp_current.index=raw_pp_current['timestamp']


In [18]:
# working_times_df['energy']=raw_pp_current[(raw_pp_current['timestamp'] <= working_times_df['end_time']) & (raw_pp_current['timestamp'] >= working_times_df['start_time'])]['total_current'].sum()*230

In [19]:
working_times_df['energy'] = working_times_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230, axis=1)

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
working_times_df.index=working_times_df.timestamp
working_times_df.drop('timestamp', axis=1, inplace=True)
working_times_df.drop('sample_number', axis=1, inplace=True)

/home/raghu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [23]:
plt.plot(test_pp["timestamp"],test_pp["boards"])
plt.plot(raw_pp_current.timestamp, raw_pp_current['total_current'])

In [25]:
PP_events=working_times_df[['event', 'working_time', 'energy']]

In [35]:
sns.distplot(PP_events.working_time)
plt.title('PP2 Working Times Histogram')

Text(0.5, 1.0, 'PP2 Working times Histogram')

In [34]:
print('PP working time mode: ', PP_events.working_time.mode().mean())

PP working time mode:  59.111111111111114
